#  English sequence generator 

Goal: Create an English language sequence generator capable of building semi-coherent English sentences from scratch by building them up character-by-character.

Dataset:  Complete version of Sir Arthur Conan Doyle's classic book The Adventures of Sherlock Holmes

Based on [RNN project: text generation](https://github.com/udacity/aind2-rnn) of the [Udacity's Artificial Intelligence  Nanodegree](https://www.udacity.com/course/artificial-intelligence-nanodegree--nd889)

In [16]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load the data

In [17]:
text = open('data/holmes.txt').read().lower()
print('Total characters: {}'.format(len(text)))
text[:300]

Characters: 581864


"\ufeffproject gutenberg's the adventures of sherlock holmes, by arthur conan doyle\n\nthis ebook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  you may copy it, give it away or\nre-use it under the terms of the project gutenberg license included\nwith this ebook or "

## Preprocess the data

In [18]:
text = text[1302:]  # remove title, author page, and table of contents
text = text.replace('\n',' ') 
text = text.replace('\r',' ')

unique_characters = set(list(text))  
print(unique_characters)

{'4', 'c', 'n', 'k', 'j', '.', 'é', '"', '7', '!', '0', 'q', 'à', 'r', '/', 'g', 'p', 'd', 'b', 'l', 'v', 'z', 'h', 'y', '9', '*', '$', 'â', 'e', 's', ':', 'm', 'i', ')', 'x', '(', '?', ',', 'f', 'è', ';', ' ', '2', 't', '6', '@', '&', 'o', 'w', 'u', '8', '3', '5', '1', 'a', "'", '-', '%'}


In [19]:
# remove non-english characters 
import re
text = re.sub("[$%&'()*@/àâèé0123456789-]", " ", text)
text = text.replace('"',' ')
text = text.replace('  ',' ') # shorten any extra dead space created above
text[:300]

'is eyes she eclipses and predominates the whole of her sex. it was not that he felt any emotion akin to love for irene adler. all emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. he was, i take it, the most perfect reasoning and observing machine '

In [21]:
chars = sorted(list(set(text)))
print('Total characters: {}'.format(len(text)))
print('Unique characters: {}'.format(len(chars)))

Total characters: 573681
Unique characters: 33
